In [ ]:
import pandas as pd
import re

# KR operations

In [ ]:
def extract_locality_and_street(input_string):
    # Список подстрок, которые могут предшествовать названию населенного пункта
    prefixes = ["г.", "г ", "с.", "с ", "кп ", "к.п.", "Город", "город"]
    locality_keyword = "по адресу:"

    # Начальные значения для города и улицы
    locality = ""
    street = input_string

    locality_index = input_string.find(locality_keyword)
    input_string = input_string[locality_index:]

    # Поиск подстроки-префикса
    for prefix in prefixes:
        index = input_string.find(prefix)
        if index != -1:
            comma_index = input_string.find(',', index)
            locality = input_string[index:comma_index].strip()
            street_start = comma_index + 1
            street = input_string[street_start:].strip()
            break

    return locality, street

In [ ]:
KR_files = ["KR on 01.01.2017 unprotected.xlsx", "KR on 01.01.2018 unprotected.xlsx", "KR on 01.01.2019 unprotected.xlsx"]

results = []

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=5) # Skip header

    columns_to_drop = [0, 2, 4, 6, 7, 10]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)

In [ ]:
results[0]

In [ ]:
results[1]

In [ ]:
results[2]

# New version sheets

In [ ]:
KR_files = ["KR on 01.01.2020.xlsx", "KR on 01.01.2021.xlsx", "KR on 01.01.2022.xlsx"]

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=2) # Skip header

    columns_to_drop = [0, 2, 4, 5, 7, 9, 10, 12, 13, 14, 15, 16, 17]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    ### Old version sheets were in thousands roubles 
    columns_to_divide = ['estimated_cost', 'approved_year_start', 'used_year_end', 'expected_year_end']
    # Convert the specified columns to numeric type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].apply(pd.to_numeric, errors='coerce')
    # Divide
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].div(1000)
    # Round the numeric values to two decimal places
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].round(2)
    # Convert the columns back to string type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].astype(str)

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)


In [ ]:
results[3]

In [ ]:
results[4]

In [ ]:
results[5]